## Testing Julia for GAIA

using the julia language for GAIA analaysis

In [27]:
using PyCall
using DataFrames

@pyimport astropy.io.votable as votable
@pyimport matplotlib.pyplot as plt

## directory
rootdir = "/home/stephane/Science/cluster/GAIA"
wdir    = "/home/stephane/Science/cluster/GAIA/products"

cd(wdir)

In [28]:
voname = "NGC 2516-1.0deg.vot"
vot = votable.parse(voname)

T = typeof(vot)
for (name, typ) in zip(fieldnames(T), T.types)
     println("type of the fieldname $name is $typ")
 end
 
b= vot[:get_first_table]() 

# c = convert(Array, b::PyObject)
#print(b)
print(keys(b::PyObject))


# print(b["array"]["data"])

for s in b["array"]["data"]
    println(s)
end


struct df
    ndata::Int32
    data::Array{Float64, Float64}
end

begin
    local ind ::Int32 = 1
    
    lgal = convert(Float64,b["array"]["data"][1]["l"])
    bgal = convert(Float64,b["array"]["data"][1]["b"])
    parallax = convert(Float64,b["array"]["data"][1]["parallax"]) 
    parallax_err = convert(Float64,b["array"]["data"][1]["parallax"]) 
    pmra  = convert(Float64,b["array"]["data"][1]["pmra"])
    pmra_err  = convert(Float64,b["array"]["data"][1]["pmra_error"])
    pmdecc  = convert(Float64,b["array"]["data"][1]["pmdec"])
    pmdec_err  = convert(Float64,b["array"]["data"][1]["pmdec_error"])
    
    G = convert(Float64,b["array"]["data"][1]["phot_g_mean_mag'"])
    Rp = convert(Float64,b["array"]["data"][1]["phot_rp_mean_mag'"])
    Bp = convert(Float64,b["array"]["data"][1]["phot_bp_mean_mag'"])
end


type of the fieldname o is Ptr{PyCall.PyObject_struct}
Symbol[:ID, :_ID, :__bytes__, :__class__, :__delattr__, :__dict__, :__dir__, :__doc__, :__eq__, :__format__, :__ge__, :__getattribute__, :__gt__, :__hash__, :__init__, :__init_subclass__, :__le__, :__lt__, :__module__, :__ne__, :__new__, :__reduce__, :__reduce_ex__, :__repr__, :__setattr__, :__sizeof__, :__str__, :__subclasshook__, :__weakref__, :_add_definitions, :_add_field, :_add_group, :_add_info, :_add_link, :_add_param, :_add_unknown_tag, :_attr_list, :_config, :_description, :_element_name, :_empty, :_fields, :_format, :_get_binary_data_stream, :_groups, :_ignore_add, :_infos, :_links, :_name, :_nrows, :_params, :_parse_binary, :_parse_fits, :_parse_tabledata, :_pos, :_ref, :_resize_strategy, :_ucd, :_ucd_in_v1_2, :_votable, :_write_binary, :_write_tabledata, :array, :create_arrays, :description, :fields, :format, :from_table, :get_field_by_id, :get_field_by_id_or_name, :get_fields_by_utype, :get_group_by_id, :get_groups_by_

PyObject (1635721458409799680, b'Gaia DR2 5289385304589191296', 5289385304589191296, 584147123, 2015.5, 117.60629089, 0.03256194, -61.12097871, 0.03030063, 0.5837983, 0.0348334, 16.75973, 5.03961048, 0.07577766, 0.47646579, 0.06326358, -0.13174221, -0.28895783, -0.20080891, 0.04546666, 0.30783668, 0.05308336, -0.07205439, 0.22346431, -0.26539093, 0.01422963, 185, 0, 182, 3, -0.94882077, 159.1197, 0., 0., 31, False, 12.336314, 1.58776399, 0.00857846, -0.16654387, 21, 15, 0.07867856, 0, 47, False, 364, 10967.29718695, 4.96795801, 2207.6067, 15.588117, 35, 5579.09316959, 10.51154509, 530.7586, 15.984979, 34, 7943.62678468, 10.48447826, 757.6559, 15.011873, 1.2330039, 0, 0.97310543, 0.39686203, 0.5762434, nan, nan, 0, nan, nan, nan, b'NOT_AVAILABLE', 273.72241661, -16.84992177, 168.79060225, -76.81159623, 100001, 5324., 5101.93, 5643.58, 0.704, 0.4846, 0.8221, 0.3478, 0.2529, 0.4041, 200111, 1.3538153, 1.204831, 1.4742342, 1.3266786, 1.1410544, 1.5123029, b'http://geadata.esac.esa.int/data

PyObject (1635721458409799680, b'Gaia DR2 5289897677007985792', 5289897677007985792, 537607294, 2015.5, 119.81161042, 0.05131767, -61.31451834, 0.05997961, 0.42473773, 0.05766946, 7.3650374, -3.31237883, 0.11466739, 7.44823387, 0.11159425, 0.06512381, 0.08986475, -0.15149851, 0.1365635, 0.35320926, 0.19415316, -0.10776446, 0.09976519, -0.10964698, 0.05256268, 220, 0, 218, 2, 6.847269, 386.89377, 0.35348469, 6.10981829, 31, False, 3.1898036, 1.54741763, 0.01507605, 0.08487517, 25, 17, 0.11046559, 0, 51, False, 446, 4992.09184861, 3.84635423, 1297.8762, 16.44266, 50, 2469.42138561, 9.87452442, 250.08003, 16.8699, 47, 3728.79542619, 8.14848732, 457.60583, 15.832998, 1.2416071, 0, 1.0369015, 0.42724037, 0.6096611, nan, nan, 0, nan, nan, nan, b'NOT_AVAILABLE', 274.40166536, -15.98951354, 172.08985903, -76.05885945, 100001, 5173.08, 5039.8867, 5361.872, nan, nan, nan, nan, nan, nan, 200111, 1.3450674, 1.2520151, 1.4171013, 1.1672922, 0.825632, 1.5089525, b'http://geadata.esac.esa.int/data-se

PyObject (1635721458409799680, b'Gaia DR2 5289895959021102208', 5289895959021102208, 904245415, 2015.5, 119.70659924, 0.0241703, -61.41957207, 0.02568029, 0.5694622, 0.0256505, 22.20082, -5.53327576, 0.053837, 5.10729003, 0.05181788, 0.05051051, 0.06881771, -0.26022023, 0.07975698, 0.23767158, 0.11539216, -0.11841374, 0.07692495, -0.17305045, 0.00951237, 227, 0, 227, 0, 0.29179236, 227.52702, 0., 0., 31, False, 13.675858, 1.56444367, 0.00684727, 0.04828145, 26, 17, 0.06041547, 0, 52, False, 451, 11289.79991441, 4.52606791, 2494.3948, 15.55665, 47, 5674.09749146, 12.53100114, 452.8048, 15.966646, 43, 8306.6919446, 11.02568489, 753.39465, 14.963349, 1.2383559, 0, 1.0032969, 0.40999603, 0.5933008, nan, nan, 0, nan, nan, nan, b'NOT_AVAILABLE', 274.47424042, -16.08280493, 172.33051728, -76.16000612, 100001, 5158.915, 4960., 5350.75, 0.76, 0.5516, 0.9083, 0.3825, 0.2912, 0.47, 200111, 1.5188321, 1.4118781, 1.6430969, 1.4721354, 1.3046228, 1.6396481, b'http://geadata.esac.esa.int/data-server/

PyObject (1635721458409799680, b'Gaia DR2 5289864141903354752', 5289864141903354752, 12444367, 2015.5, 119.13469652, 0.03730435, -61.64216874, 0.03817448, 1.49937805, 0.04044531, 37.07174, 4.20048619, 0.08312275, -18.18726362, 0.07253284, -0.09882779, -0.24674217, -0.05452643, 0.04701857, 0.29362714, 0.09788469, -0.20985878, 0.22316678, -0.07886077, -0.06100107, 212, 0, 211, 1, 13.352633, 611.88806, 0.33621526, 18.63699832, 31, False, 6.760206, 1.53552942, 0.01034481, -0.04229664, 24, 17, 0.08410311, 0, 52, False, 457, 24618.60734687, 6.64190122, 3706.5603, 14.710207, 49, 11586.86323987, 15.05217697, 769.7799, 15.191473, 49, 19086.24308173, 15.01429739, 1271.2046, 14.060119, 1.2459319, 0, 1.1313543, 0.48126602, 0.6500883, nan, nan, 0, nan, nan, nan, b'NOT_AVAILABLE', 274.55001639, -16.42715359, 172.46137547, -76.51059502, 100001, 5002.56, 4866.31, 5132.91, 0.334, 0.1889, 0.5006, 0.1782, 0.1229, 0.2731, 200111, 0.9197099, 0.873591, 0.9719321, 0.47727168, 0.44796973, 0.5065736, b'http://

...

PyObject (1635721458409799680, b'Gaia DR2 5289933411137327232', 5289933411137327232, 28488103, 2015.5, 121.12493116, 0.07173569, -61.30977612, 0.07109117, 0.49797982, 0.06985309, 7.128959, -1.79796682, 0.16381012, 4.64286596, 0.13286922, 0.03378353, -0.14007813, -0.47729993, -0.0624007, 0.09650134, -0.02852993, -0.40240604, 0.32593146, -0.01959231, 0.09243558, 210, 0, 210, 0, 2.1108265, 250.01842, 0.23847514, 1.3897127, 31, False, 2.2350078, 1.55906337, 0.01894415, -0.00788706, 24, 16, 0.15877476, 0, 49, False, 428, 2655.56838854, 1.61166953, 1647.7128, 17.127972, 43, 1261.30939303, 8.00976943, 157.47137, 17.599335, 43, 1975.37104523, 5.32790198, 370.75964, 16.522799, 1.2188277, 0, 1.0765362, 0.47136307, 0.6051731, nan, nan, 0, nan, nan, nan, b'NOT_AVAILABLE', 274.70691699, -15.43173039, 173.53716388, -75.53786273, 0, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0, nan, nan, nan, nan, nan, nan, b'http://geadata.esac.esa.int/data-server/datalink/links?ID=5289933411137327232', b'')
PyObj

PyObject (1635721458409799680, b'Gaia DR2 5289933411137327744', 5289933411137327744, 111892629, 2015.5, 121.13169615, 0.14573283, -61.30545524, 0.15293441, 1.73543294, 0.14291236, 12.143337, -14.86716201, 0.33645004, 16.08584512, 0.29123961, 0.04689496, -0.1299674, -0.4474646, -0.03140902, 0.07304818, -0.00642581, -0.41909587, 0.2561866, -0.01158559, 0.09421714, 193, 0, 193, 0, 62.737473, 5908.4917, 1.30771215, 277.65413368, 31, False, 0.5703394, 1.52667686, 0.03814919, -0.05593012, 22, 16, 0.31232947, 0, 47, False, 411, 25562.87941694, 11.22373621, 2277.573, 14.669341, 45, 13048.60451852, 15.00392013, 869.6797, 15.062478, 44, 18471.9430929, 18.14147087, 1018.2164, 14.095638, 1.2330594, 0, 0.9668398, 0.39313698, 0.5737028, nan, nan, 0, nan, nan, nan, b'NOT_AVAILABLE', 274.70460774, -15.42680456, 173.53030451, -75.532736, 100001, 5336., 5030.3335, 5423., 0.0133, 0.0053, 0.066, 0.007, 0.0026, 0.0402, 200111, 0.6916163, 0.66960335, 0.7782216, 0.34937224, 0.2909819, 0.4077626, b'http://gea

Excessive output truncated after 524313 bytes.


PyObject (1635721458409799680, b'Gaia DR2 5289873831349995136', 5289873831349995136, 1345251215, 2015.5, 120.07054305, 0.07418677, -61.70035309, 0.08086894, 0.89728645, 0.07530255, 11.9157505, -21.7520564, 0.18816895, 7.32954993, 0.15066435, 0.04899411, 0.08580849, -0.27871615, 0.08626273, 0.21050772, 0.12517762, -0.34932786, 0.0063774, 0.03564147, 0.04156825, 194, 0, 194, 0, 0.21833377, 192.60086, 0., 1.46185499e-15, 31, False, 1.9641459, 1.45351174, 0.02026524, 0.13391976, 22, 15, 0.17492875, 0, 49, False, 427, 2038.33318948, 1.54475084, 1319.5223, 17.415178, 45, 747.31647908, 8.24010392, 90.692604, 18.167627, 47, 1912.35947385, 7.24546007, 263.939, 16.557997, 1.3048289, 0, 1.6096306, 0.75244904, 0.85718155, nan, nan, 0, nan, nan, nan, b'NOT_AVAILABLE', 274.81678778, -16.05973453, 173.70947378, -76.173315, 0, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0, nan, nan, nan, nan, nan, nan, b'http://geadata.esac.esa.int/data-server/datalink/links?ID=5289873831349995136', b'')
PyObject (1

LoadError: KeyError: key "phot_g_mean_mag'" not found